In [1]:
import sys
sys.path.append('data_pre')
import json
from rouge import Rouge
import pandas as pd
import numpy as np
import copy
import os
from data_pre import title_question
from data_pre import clean_data
import utils
import pickle
import time
import gensim
from sklearn import model_selection
from config import config_base
from preprocess_data import *

config = config_base.config

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.440 seconds.
Prefix dict has been built succesfully.


In [27]:
df = pd.read_csv('data_gen/test_df.csv')

In [52]:
df[df['article_content'].isnull()]['article_content'][207] == df[df['article_content'].isnull()]['article_content'][207]

False

In [10]:
if [1]:
    print('xx')

xx


In [5]:
from jieba import posseg

In [6]:
posseg.lcut(' ', HMM=False)

[pair(' ', 'x')]

In [6]:
contents = df['article_content'].values

In [8]:
contents_fix = [c.strip() for c in contents]

In [9]:
contents_len = [len(c) for c in contents_fix]

In [11]:
min(contents_len)

0

In [12]:
questions = df['article_question'].values
question_fix = [q.strip() for q in questions]
question_len = [len(q) for q in question_fix]

In [13]:
min(question_len)

0

In [14]:
titles = df['article_title'].values
title_fix = [t.strip() for t in titles]
title_len = [len(t) for t in title_fix]

In [15]:
min(title_len)

5

In [21]:
import jieba
from jieba import posseg

In [23]:
posseg.lcut('')

[]

In [19]:
jieba.lcut('', HMM=False)

[]

In [26]:
for ss in list(''):
    print(ss)

In [24]:
tmp = ['' for s in []]

In [25]:
for ss in list('')

[]